In [1]:
import numpy as np
source1 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca.txt"
source2 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca-cls.txt"
source3 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd.txt"
source4 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd-cls.txt"
source5 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c5.txt"
source6 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c10.txt"
source7 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d5.txt"
source8 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d10.txt"
source9 = "F:/MSc DS/Project/dms/trained_dms_txt/word2dm.txt"

def get_dict_from_source(source = source5):
    dim =17
    if source ==source1 or source==source3:
        dim = 15
    whole =[]

    with open(source) as f:
        for line in f:
            linelist = line.split()
            for a in linelist:
                whole.append(a)
    chunk = dim**2+1
    n = len(whole)/chunk
    voc = []
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        a = whole[i * chunk + 1:(i + 1) * chunk]
        a = np.array(a, dtype='float')
        a.resize([dim, dim])
        value,_ = np.linalg.eig(a)
        maxeig = max(value)
        dic[voc[i]] = a/maxeig

    return dic



def kba(a, b):
    m = b-a
    eig,_ = np.linalg.eig(m)
    Kba = sum(eig)/sum(abs(eig))
    return Kba

def halfmatrix(m):
    eig,vec = np.linalg.eig(m)
    n = len(eig)
    for i in range(n):
        if eig[i]<0:
            eig[i]=0
    eignew = eig**0.5
    dim = m.shape[0]
    e = np.eye(dim)
    for i in range(dim):
        e[i][i] =eignew[i]
    vec_T = np.linalg.inv(vec)
    half = vec@e@vec_T
    return half

def compute_Kba(listA,listB, dic, method ='add'):
    '''
    imput two phrase or sentences lists
    tokenize each pair and get new dm using composition method
    compute Ke pred for each pair
    if oov set Ke to oov  # after test all in voc

    composition method
    add or pointwise
    '''

    n = len(listA)
    if method == 'add':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] + dic[phraseA[1].lower()] + dic[phraseA[2].lower()]
            newB = dic[phraseB[0].lower()] + dic[phraseB[1].lower()] + dic[phraseB[2].lower()]
            Kba_list.append(kba(newA, newB))

    if method == 'pointwise':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] * dic[phraseA[1].lower()] * dic[phraseA[2].lower()]
            newB = dic[phraseB[0].lower()] * dic[phraseB[1].lower()] * dic[phraseB[2].lower()]
            Kba_list.append(kba(newA, newB))
            
    if method == 'mult':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            s1 = dic[phraseA[0].lower()]
            v1 = dic[phraseA[1].lower()]
            o1 = dic[phraseA[2].lower()]
            s2 = dic[phraseB[0].lower()]
            v2 = dic[phraseB[1].lower()]
            o2 = dic[phraseB[2].lower()]
            newA = halfmatrix(s1)*halfmatrix(o1)*v1*halfmatrix(o1)*halfmatrix(s1)
            newB = halfmatrix(s2)*halfmatrix(o2)*v2*halfmatrix(o2)*halfmatrix(s2)
            Kba_list.append(kba(newA, newB))

    return Kba_list


# get sv dataset
dicA = []
dicB = []
val = []
with open("F:/MSc DS/Project/compositional datasets/KS2016-SVO.txt") as f:
    for line in f:
        linelist = line.split(",")
        dicA.append(linelist[0])
        dicB.append(linelist[1])
        val.append(linelist[2])


# convert the T to 1 F to 0 in val list
n = len(val)
for i in range(n):
    if val[i]=='T\n':
        val[i]=1
    else:
        val[i]=0
# dicA  list to store the first phrase
# dicB  list to store the second pharse
# val  true val


In [7]:
for i in range(len(dicA)):
    for j in dicA[i].split():
        if j.lower() not in dic1.keys():
            print(i,j)

In [6]:
for i in range(len(dicB)):
    for j in dicB[i].split():
        if j.lower() not in dic1.keys():
            print(i,j)

In [2]:
dicA.pop(22)
dicB.pop(22)
val.pop(22)
dicA.pop(22)
dicB.pop(22)
val.pop(22)

0

In [3]:
dic1 = get_dict_from_source(source1)
# add method

pred = compute_Kba(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.6498634740600714
0.6410417979416089


In [4]:
dic1 = get_dict_from_source(source2)
# add method

pred = compute_Kba(dicA, dicB,dic1)

result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.6368409997899601
0.70825456836799


In [5]:
dic1 = get_dict_from_source(source3)
# add method

pred = compute_Kba(dicA, dicB,dic1)

result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.6326402016383114
0.6851501785339215


In [6]:
dic1 = get_dict_from_source(source4)
# add method

pred = compute_Kba(dicA, dicB,dic1)

result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.5238395295106071
0.6721277042638101


In [7]:
dic1 = get_dict_from_source(source5)
# add method

pred = compute_Kba(dicA, dicB,dic1)

result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.75215290905272
0.5893719806763286


In [8]:
dic1 = get_dict_from_source(source6)
# add method

pred = compute_Kba(dicA, dicB,dic1)

result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.7347195967233775
0.7338794370930477


In [9]:
dic1 = get_dict_from_source(source7)
# add method

pred = compute_Kba(dicA, dicB,dic1)

result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.6477630749842469
0.6294896030245746


In [10]:
dic1 = get_dict_from_source(source8)
# add method

pred = compute_Kba(dicA, dicB,dic1)

result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.6536441923965555
0.5221592102499475


In [11]:
dic1 = get_dict_from_source(source9)
# add method

pred = compute_Kba(dicA, dicB,dic1)

result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
result = roc_auc_score(val, pred)
print(result)

0.621088006721277
0.44591472379752156


In [3]:
from sklearn.metrics import roc_auc_score
dic1 = get_dict_from_source(source1)

pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6395715185885319


In [4]:
dic1 = get_dict_from_source(source2)

pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.7658055030455786


In [5]:
dic1 = get_dict_from_source(source3)

pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.7032136105860115


In [6]:
dic1 = get_dict_from_source(source4)

pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6475530350766645


In [7]:
dic1 = get_dict_from_source(source5)

pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.5148078134845621


In [8]:
dic1 = get_dict_from_source(source6)

pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.7328292375551355


In [9]:
dic1 = get_dict_from_source(source7)

pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6423020373871036


In [10]:
dic1 = get_dict_from_source(source8)

pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.5421130014702793


In [11]:
dic1 = get_dict_from_source(source9)

pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.4719596723377441
